# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,troitsko-pechorsk,62.7084,56.1964,-18.89,100,94,1.08,RU,1675541509
1,1,port-cartier,50.0334,-66.8654,-19.03,69,100,8.82,CA,1675541510
2,2,san patricio,28.0170,-97.5169,20.97,52,61,4.33,US,1675541223
3,3,ushuaia,-54.8000,-68.3000,12.81,50,0,1.03,AR,1675541133
4,4,palu,-0.8917,119.8707,23.83,94,100,0.83,ID,1675541266


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 2,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500

)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df[
    (city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 23) \
    & (city_data_df["Wind Speed"] < 10) \
    & (city_data_df["Cloudiness"] < 50)                            
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,rikitea,-23.1203,-134.9692,25.85,76,3,7.04,PF,1675541189
15,15,mahebourg,-20.4081,57.7000,26.25,83,40,2.57,MU,1675540924
16,16,bambous virieux,-20.3428,57.7575,26.23,84,43,4.67,MU,1675541431
17,17,vaini,-21.2000,-175.2000,27.09,89,20,4.63,TO,1675540894
18,18,galle,6.0367,80.2170,24.96,86,23,1.65,LK,1675541511
...,...,...,...,...,...,...,...,...,...,...
514,514,koumac,-20.5667,164.2833,27.18,78,28,1.33,NC,1675541620
519,519,buchanan,5.8808,-10.0467,26.67,84,43,2.67,LR,1675541621
522,522,maceio,-9.6658,-35.7353,29.69,65,20,5.14,BR,1675541465
524,524,port hedland,-20.3167,118.5667,26.03,78,21,4.63,AU,1675541622


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
7,rikitea,PF,-23.1203,-134.9692,76,
15,mahebourg,MU,-20.4081,57.7000,83,
16,bambous virieux,MU,-20.3428,57.7575,84,
17,vaini,TO,-21.2000,-175.2000,89,
18,galle,LK,6.0367,80.2170,86,
...,...,...,...,...,...,...
514,koumac,NC,-20.5667,164.2833,78,
519,buchanan,LR,5.8808,-10.0467,84,
522,maceio,BR,-9.6658,-35.7353,65,
524,port hedland,AU,-20.3167,118.5667,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
rikitea - nearest hotel: Chez Bianca & Benoit
mahebourg - nearest hotel: Grand Bel Air
bambous virieux - nearest hotel: Otentic Eco Tent Experience
vaini - nearest hotel: Keleti International Resort
galle - nearest hotel: Jungle Hostel Villa
san cristobal - nearest hotel: Terra Sur
coihaique - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
odienne - nearest hotel: Hotel Frontière
atuona - nearest hotel: Pearl Resort
vila velha - nearest hotel: Hotel Prainha
dondo - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
rio grande - nearest hotel: Hotel Vila Moura Executivo
saint-pierre - nearest hotel: Tropic Hotel
tevaitoa - nearest hotel: Chambre d'hote
harper - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
port elizabeth - nearest hotel: Waterford Hotel
tres arroyos - nearest hotel: Andrea Hotel
canavieiras - nearest hotel: No hotel found
salalah - nearest hotel: M

,City,Country,Lat,Lng,Humidity,Hotel Name
7,rikitea,PF,-23.1203,-134.9692,76,Chez Bianca & Benoit
15,mahebourg,MU,-20.4081,57.7000,83,Grand Bel Air
16,bambous virieux,MU,-20.3428,57.7575,84,Otentic Eco Tent Experience
17,vaini,TO,-21.2000,-175.2000,89,Keleti International Resort
18,galle,LK,6.0367,80.2170,86,Jungle Hostel Villa
...,...,...,...,...,...,...
514,koumac,NC,-20.5667,164.2833,78,Monitel Koumac
519,buchanan,LR,5.8808,-10.0467,84,Phillipmena's Guesthouse
522,maceio,BR,-9.6658,-35.7353,65,Maceió Hostel e Pousada
524,port hedland,AU,-20.3167,118.5667,78,The Esplanade Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size= "Humidity",
    scale = 1,
    color = "City",
    alpha = 1,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel", "Country"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)